In [ ]:
!pip install -q -U pydantic-ai nest_asyncio

In [ ]:
from google.colab import userdata


GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
COINMARKETCAP_API_KEY=userdata.get("COINMARKETCAP_API_KEY")

In [ ]:
import json
from typing import Dict, List

from pydantic import BaseModel
from pydantic_ai import RunContext
import requests


COINMARKETCAP_API_URL = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest"

class CryptoCurrency(BaseModel):
    name: str
    price: float
    total_supply: float
    market_cap: float


def get_currencies(currency_names: List[str]) -> List[CryptoCurrency]:
    """Given a list of crypto currency names, it makes a request to the
    CoinMarketCap API and returns a list of currency objects with their
    latest details, such as prices.

    Args:
        currency_names: A list of crypto currency names to retrieve.

    Returns:
        A list of CryptoCurrency objects with their latest details.
    """
    params = {"slug": ",".join(currency_names)}
    headers = {
        "Accepts": "application/json",
        "X-CMC_PRO_API_KEY": COINMARKETCAP_API_KEY,
    }

    crypto_currencies = []
    try:
        response = requests.get(COINMARKETCAP_API_URL, params=params, headers=headers)
        data = json.loads(response.text)

        for _, currency_data in data["data"].items():
            crypto_currencies.append(
                CryptoCurrency(
                    name=currency_data["name"],
                    price=currency_data["quote"]["USD"]["price"],
                    total_supply=currency_data["total_supply"],
                    market_cap=currency_data["quote"]["USD"]["market_cap"],
                )
            )

    except Exception as exc:
        print(exc)

    return crypto_currencies

In [ ]:
import nest_asyncio
from pydantic_ai import Agent
from pydantic_ai.models.gemini import GeminiModel


nest_asyncio.apply()

model = GeminiModel(
    'gemini-2.0-flash',
    provider='google-gla',
    api_key=GEMINI_API_KEY
)

crypto_agent = Agent(
    model=model,
    result_type=str,
    tools=[
        get_currencies
    ],
    system_prompt=(
        'You are a cryptocurrency agent that can retrieve'
        'the latest details about given cruptocurrencies'
    ),
)

In [ ]:
result_sync = crypto_agent.run_sync('Give me a summary of the latest stats for monero today')
print(result_sync.data)

Here is a summary of the latest stats for Monero today:
The current price is $202.72, the market cap is $3,739,501,959.91 and the total supply is 18,446,744.07.


In [ ]:
result_sync = crypto_agent.run_sync('How much is the price?', message_history=result_sync.new_messages())
print(result_sync.data)

The current price is $202.72.
